In [9]:
import re

In [27]:
test = ["Mus musculus", "Agalma elegans", "Frillagalma vityazi","Cordagalma tottoni"]

In [40]:
second_test = ["Mus musculus (Y456)", "Agalma elegans (AB34)", "Frillagalma vityazi", "Cordagalma tottoni"]

In [38]:
for case in test:
    new_part = re.findall(r"^\w+?",case)[0] + "."
    new_string = re.sub(r"^\w+\b", new_part, case)
    print(new_string)

M. musculus
A. elegans
F. vityazi
C. tottoni


In [76]:
for case in second_test:
    new_part = re.findall(r"^\w+?",case)[0] + "."
    new_string = re.sub(r"^\w+\b", new_part, case)
    lst = re.split("[\s\(\)]",new_string)
    if len(lst) > 2:
        lst[2] = "_"
    print("".join(lst))

M.musculus_Y456
A.elegans_AB34
F.vityazi
C.tottoni


In [88]:
print(re.sub(r"([A-Z])\w+\s(\w+)\s\(([A-Z][0-9]+)\)", r"\1.\2_\3", "Mus musculus (Y456)"))

M.musculus_Y4


In [8]:
#!/usr/bin/python
import re

def convert_regexp(regexp:str):
    new_regexp = []
    transform = {'<':'^','>':'$', 'x':'.', '(':'{', ')':'}', '{':'[^','}':']', '-':'',' ':''}
    # Map PROSITE-Regexp symbols to Python Regexp-symbols
    for char in regexp:
        if char not in transform.keys():
            # If char is not metasymbol we just add char in `new_regexp` list
            new_regexp.append(char)
        else:
            # If it is then we use the char as key
            new_regexp.append(transform[char])
         # Finally, assemble new regular expression
    return "".join(new_regexp)

def get_motif(prosite:str,fasta:str):
    # This function takes PROSITE-Regexp, and FASTA FILE with one protein sequences
    regexp = convert_regexp(prosite) # Transform to PythonRegexp form
    with open(fasta) as file:
        name = file.readline() # First line include AC number. It's need in answer
        FASTA_ID = re.findall(r">\w+", name)[0] # Extract AC number
        seq = "".join(map(lambda x: x.strip(), file.readlines())) # It's sequence of protein
        motif = re.search(regexp,seq) # find motif (KKCGHM)
        span = motif.span() 
        # Return variables in dictionary for further work
        return {"seq":seq, "ID":FASTA_ID, "motif":motif.group(), "span":span}
    
result = get_motif("K-[KR]-C-G-H-[LMQR]","isocitrate_lyase.fasta")
# prefix is first "zzz" in pattern
prefix = result["seq"][result["span"][0]-3:result["span"][0]].lower()
# suffix is last one
suffix = result["seq"][result["span"][1]:result["span"][1]+3].lower()
context_motif = prefix+result["motif"]+suffix
# We have extracted span to show start poistion
N = str(result["span"][0])
# Print answer like task`s pattern.
print("Isocitrate lyase (" + result["ID"] + ") contains its active site PS00161 starting from "+ N + "th position: " + context_motif)

Isocitrate lyase (>WP_000857856) contains its active site PS00161 starting from 192th position: asvKKCGHMggk
